In [3]:
import pandas as pd
import numpy as np

In [59]:
ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
TEST_PATH = r'../../data/raw/test.csv'
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

In [15]:
import os
import random
import numpy as np
import pandas as pd
import joblib
from pathlib import Path
from datetime import datetime


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def save_model(model, filename, directory='models'):
    from .config import MODELS_DIR
    
    filepath = MODELS_DIR / filename
    joblib.dump(model, filepath)
    print(f"Model saved to: {filepath}")
    return str(filepath)


def load_model(filename, directory='models'):
    from .config import MODELS_DIR
    
    filepath = MODELS_DIR / filename
    model = joblib.load(filepath)
    print(f"Model loaded from: {filepath}")
    return model


def save_submission(predictions, student_ids, team_name, directory='output'):
    from .config import OUTPUT_DIR
    
    submission = pd.DataFrame({
        'MA_SO_SV': student_ids,
        'PRED_TC_HOANTHANH': predictions.astype(int)
    })
    
    filename = f"{team_name}.csv"
    filepath = OUTPUT_DIR / filename
    submission.to_csv(filepath, index=False)
    print(f"Submission saved to: {filepath}")
    return str(filepath)


def create_semester_code(year, semester):
    next_year = year + 1
    return f"HK{semester} {year}-{next_year}"


def parse_semester_code(semester_code):
    parts = semester_code.strip().split()
    semester = int(parts[0].replace('HK', ''))
    year_range = parts[1].split('-')
    year = int(year_range[0])
    return year, semester


def get_semester_order(semester_code):
    year, semester = parse_semester_code(semester_code)
    return year * 10 + semester


def calculate_semester_from_admission(admission_year, current_semester_code):
    current_year, current_sem = parse_semester_code(current_semester_code)
    years_diff = current_year - admission_year
    return years_diff * 2 + current_sem


def log_experiment(experiment_name, metrics, params, directory='output'):
    from .config import OUTPUT_DIR
    import pandas as pd
    from datetime import datetime

    log_file = OUTPUT_DIR / 'experiment_log.csv'

    log_entry = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'experiment': experiment_name,
        **metrics,
        **{f'param_{k}': v for k, v in params.items()}
    }

    if log_file.exists() and log_file.stat().st_size > 0:
        log_df = pd.read_csv(log_file)
        log_df = pd.concat(
            [log_df, pd.DataFrame([log_entry])],
            ignore_index=True
        )
    else:
        log_df = pd.DataFrame([log_entry])

    log_df.to_csv(log_file, index=False)
    print(f"Experiment logged to: {log_file}")

    return log_file

def memory_usage(df):
    memory_mb = df.memory_usage(deep=True).sum() / 1024**2
    return f"{memory_mb:.2f} MB"

In [5]:
import pandas as pd
import numpy as np

def clean_data_pipeline(admission, academic_records):
    print("--- BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU ---")
    
    # =========================================================================
    # BƯỚC 1: XỬ LÝ ĐỊNH DẠNG & KIỂU DỮ LIỆU (Formatting)
    # =========================================================================
    print("Step 1: Formatting & Merging...")
    
    # 1.1. Chuẩn hóa MA_SO_SV về dạng String
    admission['MA_SO_SV'] = admission['MA_SO_SV'].astype(str)
    academic_records['MA_SO_SV'] = academic_records['MA_SO_SV'].astype(str)
    
    # 1.2. Chuyển đổi HOC_KY sang số nguyên (giả định hàm hoc_ky_to_code đã được định nghĩa bên ngoài)
    # Nếu chưa có hàm này, bạn cần định nghĩa logic convert (ví dụ: '20231' -> int)
    # academic_records['HOC_KY'] = academic_records['HOC_KY'].apply(hoc_ky_to_code)
    
    # Merge dữ liệu (Inner Join để chỉ lấy sinh viên có thông tin ở cả 2 bảng)
    df = pd.merge(academic_records, admission, on='MA_SO_SV', how='inner')
    
    # 1.3. Định dạng lại các cột số (Ép kiểu float/int)
    numeric_floats = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    for col in numeric_floats:
        df[col] = pd.to_numeric(df[col], errors='coerce') 
        
    numeric_ints = ['TC_DANGKY', 'TC_HOANTHANH']
    for col in numeric_ints:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Sắp xếp dữ liệu
    df = df.sort_values(by=['MA_SO_SV', 'HOC_KY']).reset_index(drop=True)

    # =========================================================================
    # BƯỚC 2: KIỂM TRA LOGIC & LÀM SẠCH NHIỄU (Sanity Checks)
    # =========================================================================
    print("Step 2: Sanity Checks & Logic Cleaning...")
    
    initial_rows = len(df)

    # 3.1. Logic Tín chỉ: Hoàn thành không được lớn hơn Đăng ký
    df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
    
    # 3.2. Logic Điểm số (0 <= GPA/CPA <= 4.0)
    df['GPA'] = df['GPA'].clip(lower=0.0, upper=4.0)
    df['CPA'] = df['CPA'].clip(lower=0.0, upper=4.0)
    
    # -----------------------------------------------------------
    # [MỚI] 3.3. Logic Tuyển sinh: DIEM_TRUNGTUYEN >= DIEM_CHUAN
    # -----------------------------------------------------------
    # Loại bỏ các dòng mà điểm trúng tuyển nhỏ hơn điểm chuẩn.
    # Lưu ý: Các giá trị NaN (do lỗi format ở B1) cũng sẽ bị loại bỏ trong phép so sánh này.
    rows_before_score_filter = len(df)
    df = df[df['DIEM_TRUNGTUYEN'] >= df['DIEM_CHUAN']]
    dropped_score_rows = rows_before_score_filter - len(df)
    print(f" -> Đã loại bỏ {dropped_score_rows} dòng do Điểm trúng tuyển < Điểm chuẩn.")

    # 3.4. Xóa dữ liệu rác (TC_DANGKY = 0)
    rows_before_credit_filter = len(df)
    df = df[df['TC_DANGKY'] > 0].copy()
    dropped_credit_rows = rows_before_credit_filter - len(df)
    print(f" -> Đã loại bỏ {dropped_credit_rows} dòng rác (TC_DANGKY=0).")
    
    # Tổng kết
    total_dropped = initial_rows - len(df)
    print(f"--- HOÀN TẤT: Tổng cộng đã loại bỏ {total_dropped} dòng nhiễu. Kích thước data cuối: {df.shape} ---")
    
    return df

In [19]:
df = clean_data_pipeline(admission, academic_records)

--- BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU ---
Step 1: Formatting & Merging...
Step 2: Sanity Checks & Logic Cleaning...
 -> Đã loại bỏ 0 dòng do Điểm trúng tuyển < Điểm chuẩn.
 -> Đã loại bỏ 0 dòng rác (TC_DANGKY=0).
--- HOÀN TẤT: Tổng cộng đã loại bỏ 0 dòng nhiễu. Kích thước data cuối: (105726, 11) ---


In [20]:
df.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,2023,100,A00,21.32,20.25
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,2023,100,A00,21.32,20.25
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,2021,1,D07,23.84,22.43
3,000e15519006,HK1 2022-2023,2.83,2.98,21,21,2021,1,D07,23.84,22.43
4,000e15519006,HK1 2023-2024,1.50,2.73,20,14,2021,1,D07,23.84,22.43


In [11]:
def split_data(merged_df, train_end='HK1 2023-2024', valid_semester='HK2 2023-2024'):
    print("Splitting data into train and validation sets...")
    merged_df['semester_order'] = merged_df['HOC_KY'].apply(get_semester_order)
    train_end_order = get_semester_order(train_end)
    valid_order = get_semester_order(valid_semester)
    train_df = merged_df[merged_df['semester_order'] <= train_end_order].copy()
    valid_df = merged_df[merged_df['semester_order'] == valid_order].copy()    
    print(f"Train data: {train_df.shape}")
    print(f"Valid data: {valid_df.shape}")
        
    return train_df, valid_df

In [17]:
train, valid = split_data(df)

Splitting data into train and validation sets...
Train data: (90582, 12)
Valid data: (15144, 12)


In [18]:
train.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,semester_order
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,2023,100,A00,21.32,20.25,20231
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,2021,1,D07,23.84,22.43,20211
3,000e15519006,HK1 2022-2023,2.83,2.98,21,21,2021,1,D07,23.84,22.43,20221
4,000e15519006,HK1 2023-2024,1.50,2.73,20,14,2021,1,D07,23.84,22.43,20231
5,000e15519006,HK2 2021-2022,2.77,3.12,19,19,2021,1,D07,23.84,22.43,20212


In [42]:
def get_feature_columns(df, exclude_cols=None):
        if exclude_cols is None:
            exclude_cols = [
                'MA_SO_SV',
                'HOC_KY',
                'semester_order',
                'completion_rate',
                'tc_failed',
                'total_tc_completed',
                'avg_completion_rate',
                'completion_rate_trend'
            ]
        
        feature_cols = [col for col in df.columns if col not in exclude_cols]
        
        categorical_cols = df[feature_cols].select_dtypes(include=['object']).columns.tolist()
        
        return feature_cols, categorical_cols

In [43]:
feature_cols, categorical_cols = get_feature_columns(train)

In [44]:
feature_cols

['CPA',
 'GPA',
 'TC_DANGKY',
 'TC_HOANTHANH',
 'NAM_TUYENSINH',
 'PTXT',
 'TOHOP_XT',
 'DIEM_TRUNGTUYEN',
 'DIEM_CHUAN']

In [45]:
categorical_cols

['PTXT', 'TOHOP_XT']

In [47]:
target_col='TC_HOANTHANH'

In [48]:
X_train = train[feature_cols]
X_valid = valid[feature_cols]
y_train = train[target_col]
y_valid = valid[target_col]

In [50]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_features(X, categorical_cols, label_encoders=None, is_training=True):
    X = X.copy()
    if label_encoders is None:
        label_encoders = {}
    for col in categorical_cols:
        if col in X.columns:
            if is_training:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))
                label_encoders[col] = le
            else:
                le = label_encoders.get(col)
                if le is not None:
                    # Xử lý giá trị mới chưa từng xuất hiện
                    X[col] = X[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
                    if 'Unknown' not in le.classes_:
                        le.classes_ = np.append(le.classes_, 'Unknown')
                    X[col] = le.transform(X[col])
    return X, label_encoders

In [51]:
# Encode train
X_train_enc, label_encoders = encode_categorical_features(X_train, categorical_cols, is_training=True)
# Encode valid
X_valid_enc, _ = encode_categorical_features(X_valid, categorical_cols, label_encoders, is_training=False)

In [55]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train_enc, y_train, eval_set=[(X_valid_enc, y_valid)], verbose=True)


[0]	validation_0-rmse:4.97792
[1]	validation_0-rmse:3.48635
[2]	validation_0-rmse:2.44091
[3]	validation_0-rmse:1.70953
[4]	validation_0-rmse:1.19763
[5]	validation_0-rmse:0.83900
[6]	validation_0-rmse:0.58783
[7]	validation_0-rmse:0.41188
[8]	validation_0-rmse:0.28871
[9]	validation_0-rmse:0.20250
[10]	validation_0-rmse:0.14205
[11]	validation_0-rmse:0.09982
[12]	validation_0-rmse:0.07052
[13]	validation_0-rmse:0.05024
[14]	validation_0-rmse:0.03631
[15]	validation_0-rmse:0.02640
[16]	validation_0-rmse:0.02017
[17]	validation_0-rmse:0.01619
[18]	validation_0-rmse:0.01356
[19]	validation_0-rmse:0.01129
[20]	validation_0-rmse:0.01043
[21]	validation_0-rmse:0.00942
[22]	validation_0-rmse:0.00874
[23]	validation_0-rmse:0.00877
[24]	validation_0-rmse:0.00852
[25]	validation_0-rmse:0.00806
[26]	validation_0-rmse:0.00777
[27]	validation_0-rmse:0.00763
[28]	validation_0-rmse:0.00695
[29]	validation_0-rmse:0.00680
[30]	validation_0-rmse:0.00671
[31]	validation_0-rmse:0.00663
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [60]:
test_df = pd.read_csv(TEST_PATH)


In [61]:
test_df

,MA_SO_SV,HOC_KY,TC_DANGKY
0,481436e2064d,HK1 2024-2025,3
1,6c8a97d22131,HK1 2024-2025,3
2,e87f62beabbb,HK1 2024-2025,13
3,438aff5ef524,HK1 2024-2025,1
4,ad172a9b0722,HK1 2024-2025,17
...,...,...,...
16497,9e803a0d26f0,HK1 2024-2025,50
16498,dbc819721795,HK1 2024-2025,56
16499,9e1c8deafb70,HK1 2024-2025,49
16500,ffecfc70f83a,HK1 2024-2025,51


In [67]:
def create_test_features(test_df, train_df):
        # train_df = create_features(train_df, is_training=True)
        train_df = train_df.sort_values(['MA_SO_SV', 'semester_order'])
        last_records = train_df.groupby('MA_SO_SV').last().reset_index()
        
        admission_info = train_df[['MA_SO_SV', 'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 
                                   'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']].drop_duplicates('MA_SO_SV')
        
        test_features = test_df.merge(
            admission_info,
            on='MA_SO_SV',
            how='left'
        )
        
        test_features = test_features.merge(
            last_records[['MA_SO_SV', 'CPA', 'GPA',
                         'total_tc_completed_lag1', 'avg_completion_rate_lag1',
                         'avg_gpa_lag1', 'num_previous_semesters']],
            on='MA_SO_SV',
            how='left',
            suffixes=('', '_last')
        )
        
        test_features['CPA'] = test_features['CPA'].fillna(0)
        test_features['GPA'] = test_features['GPA'].fillna(0)
        test_features['total_tc_completed_lag1'] = test_features['total_tc_completed_lag1'].fillna(0)
        test_features['avg_completion_rate_lag1'] = test_features['avg_completion_rate_lag1'].fillna(0)
        test_features['avg_gpa_lag1'] = test_features['avg_gpa_lag1'].fillna(0)
        test_features['num_previous_semesters'] = test_features['num_previous_semesters'].fillna(0)
        
        # test_features = self.create_features(test_features, is_training=False)
        
        return test_features

In [68]:
# 1. Tạo test_features (giả sử bạn đã có hàm create_test_features)
test_features = create_test_features(test_df, train)

# 2. Lấy đúng các cột feature đã dùng khi train
X_test = test_features[feature_cols]

# 3. Encode categorical features giống train
X_test_enc, _ = encode_categorical_features(X_test, categorical_cols, label_encoders, is_training=False)

# 4. Dự đoán
y_test_pred = model.predict(X_test_enc)

KeyError: "['total_tc_completed_lag1', 'avg_completion_rate_lag1', 'avg_gpa_lag1', 'num_previous_semesters'] not in index"

In [62]:
y_pred = model.predict(test_df)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:MA_SO_SV: object, HOC_KY: object

In [71]:
# 1. Định nghĩa lại feature_cols (LOẠI BỎ 'TC_HOANTHANH')
feature_cols = [
    'CPA', 'GPA', 'TC_DANGKY', 
    'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 
    'DIEM_TRUNGTUYEN', 'DIEM_CHUAN'
]
categorical_cols = ['PTXT', 'TOHOP_XT']

# 2. Chuẩn bị dữ liệu train
X_train = train[feature_cols]
X_valid = valid[feature_cols]
y_train = train['TC_HOANTHANH']  # Target riêng biệt
y_valid = valid['TC_HOANTHANH']

# 3. Encode (Dùng hàm có sẵn của bạn)
X_train_enc, label_encoders = encode_categorical_features(X_train, categorical_cols, is_training=True)
X_valid_enc, _ = encode_categorical_features(X_valid, categorical_cols, label_encoders, is_training=False)

# 4. Train lại model
import xgboost as xgb
model = xgb.XGBRegressor(enable_categorical=True) # Bật tính năng hỗ trợ category nếu cần
model.fit(X_train_enc, y_train, eval_set=[(X_valid_enc, y_valid)], verbose=True)
print("Train xong!")

[0]	validation_0-rmse:5.15918
[1]	validation_0-rmse:3.79623
[2]	validation_0-rmse:2.89506
[3]	validation_0-rmse:2.31510
[4]	validation_0-rmse:1.95524
[5]	validation_0-rmse:1.73103
[6]	validation_0-rmse:1.59947
[7]	validation_0-rmse:1.52419
[8]	validation_0-rmse:1.48293
[9]	validation_0-rmse:1.46034
[10]	validation_0-rmse:1.44306
[11]	validation_0-rmse:1.43544
[12]	validation_0-rmse:1.42943
[13]	validation_0-rmse:1.42778
[14]	validation_0-rmse:1.42824
[15]	validation_0-rmse:1.42979
[16]	validation_0-rmse:1.43053
[17]	validation_0-rmse:1.42990
[18]	validation_0-rmse:1.42918
[19]	validation_0-rmse:1.42997
[20]	validation_0-rmse:1.43181
[21]	validation_0-rmse:1.43184
[22]	validation_0-rmse:1.43220
[23]	validation_0-rmse:1.43251
[24]	validation_0-rmse:1.43412
[25]	validation_0-rmse:1.43403
[26]	validation_0-rmse:1.43443
[27]	validation_0-rmse:1.43472
[28]	validation_0-rmse:1.43487
[29]	validation_0-rmse:1.43529
[30]	validation_0-rmse:1.43509
[31]	validation_0-rmse:1.43547
[32]	validation_0-

In [72]:
def prepare_test_data(test_df, train_df):
    # 1. Lấy thông tin tĩnh (Admission Info) từ train
    # Mỗi sinh viên chỉ lấy 1 dòng thông tin tuyển sinh
    static_cols = ['MA_SO_SV', 'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    admission_info = train_df[static_cols].drop_duplicates('MA_SO_SV')
    
    # 2. Lấy thông tin học lực mới nhất (Latest Performance)
    # Sắp xếp theo kỳ học để lấy dòng cuối cùng (trạng thái mới nhất của SV trước khi vào kỳ test)
    train_sorted = train_df.sort_values(['MA_SO_SV', 'semester_order'])
    last_performance = train_sorted.groupby('MA_SO_SV')[['CPA', 'GPA']].last().reset_index()
    
    # 3. Merge vào test_df
    # Merge thông tin tuyển sinh
    final_test = test_df.merge(admission_info, on='MA_SO_SV', how='left')
    
    # Merge thông tin CPA/GPA
    final_test = final_test.merge(last_performance, on='MA_SO_SV', how='left')
    
    # 4. Xử lý thiếu dữ liệu (cho sinh viên mới hoặc dữ liệu bị thiếu)
    final_test['CPA'] = final_test['CPA'].fillna(0)
    final_test['GPA'] = final_test['GPA'].fillna(0)
    
    # Lưu ý: Nếu bạn có dùng feature lag (như code cũ), bạn phải tính toán lag ở bước này. 
    # Nhưng vì code train hiện tại chưa có lag, nên ta bỏ qua để tránh lỗi KeyError.
    
    return final_test

# Tạo dữ liệu test đầy đủ
df_test_full = prepare_test_data(test_df, train)
print(f"Kích thước tập test sau khi merge: {df_test_full.shape}")
df_test_full.head()

Kích thước tập test sau khi merge: (16502, 10)


,MA_SO_SV,HOC_KY,TC_DANGKY,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,CPA,GPA
0,481436e2064d,HK1 2024-2025,3,2019.0,1,A00,22.79,19.71,1.70,2.10
1,6c8a97d22131,HK1 2024-2025,3,2018.0,1,A00,20.72,18.79,2.96,1.95
2,e87f62beabbb,HK1 2024-2025,13,2020.0,1,A00,23.09,19.37,1.51,1.71
3,438aff5ef524,HK1 2024-2025,1,2018.0,1,A00,17.53,16.12,0.17,1.85
4,ad172a9b0722,HK1 2024-2025,17,2020.0,1,A00,25.58,22.16,1.65,2.30


In [77]:
# 1. Lấy đúng các cột feature (phải khớp thứ tự với lúc train)
X_test = df_test_full[feature_cols]

# 2. Encode (QUAN TRỌNG: is_training=False để dùng lại bộ encoder cũ)
X_test_enc, _ = encode_categorical_features(X_test, categorical_cols, label_encoders, is_training=False)

# 3. Dự đoán
y_pred = model.predict(X_test_enc)

# 4. Làm tròn và chặn trên (không được vượt quá số tín chỉ đăng ký)
# Ví dụ: Dự đoán 19.5 -> 20, nhưng nếu đăng ký 18 thì chỉ được max là 18
import numpy as np
final_preds = y_pred
final_preds = np.minimum(final_preds, df_test_full['TC_DANGKY'])
final_preds = np.maximum(final_preds, 0) # Không được âm

# 5. Lưu kết quả
submission = pd.DataFrame({
    'MA_SO_SV': test_df['MA_SO_SV'],
    'PRED_TC_HOANTHANH': final_preds.astype(float)
})

submission.to_csv('submission.csv', index=False)
print("Đã lưu file submission.csv thành công!")

Đã lưu file submission.csv thành công!
